In [68]:
import sys
import os

PROJECT_ROOT = os.path.abspath(os.path.join(os.getcwd(), ".."))
if PROJECT_ROOT not in sys.path:
    sys.path.insert(0, PROJECT_ROOT)

from src.models.decision_tree import DecisionTreeModel
from src.dataset import Dataset

In [77]:
dataset = Dataset()
f_train, f_test, l_train, l_test = dataset.split_data(target_column = 'LUNG_CANCER')

In [82]:
model = DecisionTreeModel(
    max_depth=7,
    min_samples_leaf=3
)

model.train(f_train, l_train)

In [83]:
print(model.score(f_train, l_train))
print(model.score(f_test, l_test))

0.9409090909090909
0.875
